In [1]:
import pickle
from neo4j import GraphDatabase
from dotenv import dotenv_values
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from cdlib import algorithms, viz, evaluation

from hp_nlp_graph.neo4j import add_metrics_to_neo4j

Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'graph_tool', 'infomap', 'bayanpy'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'pyclustering', 'ASLPAw'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap'}


In [2]:
interactions_by_chapter = pickle.load(open("data/interactions_by_chapter.pkl", "rb"))

In [3]:
neo4j_config = dotenv_values(".env")
driver = GraphDatabase.driver(
    neo4j_config["NEO4J_URL"],
    auth=(neo4j_config["NEO4J_USER"], neo4j_config["NEO4J_PASSWORD"]),
)

In [4]:
dfs = []
for chapter, distances in interactions_by_chapter.items():
    df = pd.DataFrame.from_dict(
        dict(distances), orient="index", columns=["weight"]
    ).reset_index(names=["characters"])
    df[["source", "target"]] = pd.DataFrame(df.characters.tolist(), index=df.index)
    df["chapter"] = chapter
    df["book"] = 1
    df = df[["source", "target", "weight", "chapter", "book"]]
    dfs.append(df)
df = (
    pd.concat(dfs)
    .groupby(["source", "target"])
    .weight.sum()
    .sort_values(ascending=False)
    .reset_index()
)

In [5]:
G = nx.from_pandas_edgelist(df, "source", "target", ["weight"], create_using=nx.Graph())

In [6]:
eigen_centrality = nx.eigenvector_centrality(G, weight="weight")
betweenness_centrality = nx.betweenness_centrality(G, weight="weight")
degree_centrality = nx.degree_centrality(G)
closeness_centrality = nx.closeness_centrality(G)
pagerank = nx.pagerank(G, weight="weight")
hub_centrality, authority_centrality = nx.hits(G)
degree = dict(nx.degree(G))
weighted_degree = dict(nx.degree(G, weight="weight"))

metrics = {
    "eigen_centrality": eigen_centrality,
    "betweenness_centrality": betweenness_centrality,
    "degree_centrality": degree_centrality,
    "closeness_centrality": closeness_centrality,
    "pagerank": pagerank,
    "hub": hub_centrality,
    "authority": authority_centrality,
    "degree": degree,
    "weighted_degree": weighted_degree,
}
metrics_df = pd.DataFrame.from_dict(metrics)
metrics_df.index.name = "name"
metrics_df

,eigen_centrality,betweenness_centrality,degree_centrality,closeness_centrality,pagerank,hub,authority,degree,weighted_degree
name,,,,,,,,,
Harry Potter,6.180255e-01,0.314541,0.669421,0.717884,0.203707,1.998641e-01,1.998641e-01,81,3726
Ronald Weasley,5.111887e-01,0.114445,0.454545,0.573527,0.102588,1.653188e-01,1.653188e-01,55,1982
Hermione Granger,4.424195e-01,0.058509,0.305785,0.519847,0.076365,1.430776e-01,1.430776e-01,37,1559
Rubeus Hagrid,2.897066e-01,0.060440,0.305785,0.532974,0.052222,9.369125e-02,9.369125e-02,37,1052
Severus Snape,1.489497e-01,0.021739,0.214876,0.488560,0.029145,4.817020e-02,4.817020e-02,26,594
...,...,...,...,...,...,...,...,...,...
Ptolemy,7.542469e-04,0.020919,0.024793,0.420434,0.001556,2.439220e-04,2.439220e-04,3,3
Emeric the Evil,8.441879e-06,0.000000,0.016529,0.311295,0.002291,2.730063e-06,2.730063e-06,2,2
Cliodna,4.122414e-04,0.000000,0.016529,0.409027,0.001966,1.333208e-04,1.333208e-04,2,2


In [7]:
louvain_comms = algorithms.louvain(G, weight="weight")
louvain = pd.DataFrame.from_dict(
    dict(louvain_comms.to_node_community_map()),
    orient="index",
    columns=["louvain"],
)
louvain.index.name = "name"

In [8]:
leiden_comms = algorithms.leiden(G, weights="weight")
leiden = pd.DataFrame.from_dict(
    dict(leiden_comms.to_node_community_map()),
    orient="index",
    columns=["leiden"],
)
leiden.index.name = "name"

In [9]:
girvan_newman_comms = algorithms.girvan_newman(G, level=5)
girvan_newman = pd.DataFrame.from_dict(
    dict(girvan_newman_comms.to_node_community_map()),
    orient="index",
    columns=["girvan_newman"],
)
girvan_newman.index.name = "name"

In [10]:
spectral_comms = algorithms.spectral(G, kmax=8)
spectral = pd.DataFrame.from_dict(
    dict(spectral_comms.to_node_community_map()),
    orient="index",
    columns=["spectral"],
)
spectral.index.name = "name"

In [11]:
metrics_df = metrics_df.join([louvain, leiden, girvan_newman, spectral])

In [12]:
metrics_df.to_csv("data/metrics.csv")

In [13]:
add_metrics_to_neo4j(driver, metrics_df.reset_index().to_dict("records"))

In [14]:
for col in metrics_df.columns:
    nx.set_node_attributes(G, metrics_df[col].to_dict(), col)

In [15]:
nx.write_gexf(G, "data/graph.gexf")

In [17]:
df.sum()

source    Harry PotterHarry PotterHarry PotterHermione G...
target    Ronald WeasleyHermione GrangerRubeus HagridRon...
weight                                                 6972
dtype: object